In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd

def find_duplicate_words(title, stop_words):
    words = title.split()
    word_counts = {}
    duplicate_words = []

    
    for word in words:
        if word in word_counts and word not in stop_words:
            word_counts[word] += 1
            if word_counts[word] == 2:
                duplicate_words.append(word)
        else:
            word_counts[word] = 1

    return duplicate_words

# Функція для обробки файлу та відображення результатів у вікні
def parse_file():
    # Відкрити діалогове вікно для вибору файлу з URL та title
    file_path = filedialog.askopenfilename(filetypes=[("Excel Files", "*.xlsx")])

    if not file_path:
        return  # Користувач скасував вибір файлу

    # Зчитуємо дані з файлу xlsx
    try:
        df = pd.read_excel(file_path)
    except Exception as e:
        result_label.config(text=f"Помилка при зчитуванні файлу: {str(e)}")
        return

    # Створюємо порожні списки для збереження результатів
    urls_with_duplicates = []
    titles_with_duplicates = []

    # Список стоп-слів, які програма не сприйматиме як дублікати
    stop_words = ['-', ',', '.', 'The','the', 'and', '!', '?']  # Замініть це на ваш список стоп-слів

   
    for index, row in df.iterrows():
        title = row['Title']
        url = row['Address']

        # Знаходимо дублікати слів у Title
        duplicate_words = find_duplicate_words(title, stop_words)

        # Якщо є дублікати слів у Title, і ці слова не є стоп-словами, то додаємо URL та Title у результати
        if duplicate_words:
            urls_with_duplicates.append(url)
            titles_with_duplicates.append(title)

    # Створюємо DataFrame із списками URL та Title
    result_df = pd.DataFrame({'URL': urls_with_duplicates, 'Title': titles_with_duplicates})

    # Виводимо попередження у вікні
    if not result_df.empty:
        result_text = "Файл успішно оброблено і результати можна завантажити, оберіть шлях для завантаження\n\n" 
        messagebox.showinfo("Успішно", result_text)

        # Запит на вибір шляху для збереження файлу результатів
        save_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel Files", "*.xlsx")])

        if save_path:
            try:
                result_df.to_excel(save_path, index=False)
                messagebox.showinfo("Успішно", f"Результати збережено у файлі: {save_path}")
            except Exception as e:
                messagebox.showerror("Помилка", f"Помилка при збереженні файлу: {str(e)}")
    else:
        messagebox.showinfo("Інформація", "Дублікати слів не знайдено у жодному Title.")


root = tk.Tk()
root.title("Пошук дублів слів у title")


root.geometry("400x200")

# Додаємо кнопку "Завантажити файл" для обрання файлу
load_button = tk.Button(root, text="Завантажити файл", command=parse_file)
load_button.pack()

root.mainloop() 
